In [114]:
import sys
import os
import numpy as np
import time
# Add the parent folder to sys.path
sys.path.insert(0, os.path.abspath(os.path.join(os.getcwd(), '..')))

import pandas as pd
import density_generator
import numpy as np
import matplotlib.pyplot as plt
import qutip
import importlib
import helper_functions
import quantum_subroutines_pennylane
import pipeline_v200
import vqfe_subroutine
import pennylane as qml
import circuit_generator
from scipy.linalg import eigh
#from pennylane import numpy as np

from random import randint
importlib.reload(density_generator)
importlib.reload(helper_functions)
importlib.reload(quantum_subroutines_pennylane)
importlib.reload(vqfe_subroutine)
importlib.reload(pipeline_v200)
importlib.reload(circuit_generator)

pass;

In [115]:
## parameters
N=4
n=3
#trace_out_indices= list(range(N - n))
trace_out_indices = np.random.choice(range(N), size= N - n, replace=False)
#trace_out_indices = [0]
J=5
h_z=0.1
delta=0.1
m=1
# initial_state=None
DEBUG=False
derivative_delta=1e-3

print("N =",N, " starting number of qubits, before tracing out")
print("n =",n," final number of qubits")
print("trace_out_indices =",trace_out_indices)
print("J =",J)
print("h_z =",h_z)
print("delta =",delta)
print("m =",m)
print("DEBUG is set to: ",DEBUG)

N = 4  starting number of qubits, before tracing out
n = 3  final number of qubits
trace_out_indices = [2]
J = 5
h_z = 0.1
delta = 0.1
m = 1
DEBUG is set to:  False


In [116]:
trace_out_indices

array([2])

In [117]:
# classical
print("TEST 1 : classical\n")
model = density_generator.SpinChainXXZ(n=N, J=J, h_z=h_z, initial_state="0", DEBUG=DEBUG) 
## using N because i will trace out

rho_pure, rho_delta_pure = model.generate_density_matrices_with_perturbation(delta=delta)
print("rho_pure shape = ", rho_pure.shape)
evals = np.linalg.eigvalsh(rho_pure)
entropy = -np.sum(evals * np.log2(evals + 1e-12))  # small epsilon for numerical stability
print("entanglement entropy = ", entropy)


rho_mixed, rho_delta_mixed = model.generate_mixed_density_matrices_with_perturbation(delta=delta,trace_out_indices=trace_out_indices)
print("rho_mixed shape = ", rho_mixed.shape)
print("purity of rho", np.trace(rho_mixed @ rho_mixed))
print("purity of rho_delta", np.trace(rho_delta_mixed @ rho_delta_mixed))

evals = np.linalg.eigvalsh(rho_mixed)
entropy = -np.sum(evals * np.log2(evals + 1e-12))  # small epsilon for numerical stability
print("entanglement entropy = ", entropy)

print("uhlmann fidelity = ", helper_functions.uhlmann_fidelity_root(rho_mixed, rho_delta_mixed))
print("fidelity pennylane = ",helper_functions.fidelity_pennylane(rho_mixed, rho_delta_mixed))


TEST 1 : classical

rho_pure shape =  (16, 16)
entanglement entropy =  -1.438257255077022e-12
rho_mixed shape =  (8, 8)
purity of rho (0.7445570639566963+0j)
purity of rho_delta (0.7350848259640541+0j)
entanglement entropy =  0.6106318446709024
uhlmann fidelity =  0.9979352759614042
fidelity pennylane =  0.9958748300808027


In [120]:
print("TEST 2 : quantum\n")

DRAW = False

import pennylane as qml
import pennylane.numpy as np

circuit_fn = circuit_generator.make_ising_theta_and_theta_plus_delta_circuits(J, h_z, delta, n) 

# Attach to a device and QNode
dev = qml.device("default.qubit", wires=2 * n)

# trace out for the whole system composed by sys 1 and sys 2

active_rho = [x for x in list(range(n)) if x not in trace_out_indices] 
active_rho_delta = [x + n for x in active_rho]

measured_wires = active_rho + active_rho_delta
discarded_wires = [x for x in range(2*n) if x not in measured_wires]

TEST 2 : quantum



In [123]:
print(active_rho)
print(active_rho_delta)

[0, 1, 2]
[3, 4, 5]


In [ ]:


@qml.qnode(dev)
def qnode():
    circuit_fn()
    #return qml.state()
    return qml.density_matrix(wires=measured_wires)



# --- Print the Circuit ---
wire_options = {
    #i: {'color': 'teal', 'linestyle': '--'} for i in discaded_wires # discarded wires
    i: {'linestyle': '--'} for i in discarded_wires # discarded wires
}

if DRAW:
    # Draw the circuit
    drawer = qml.draw_mpl(qnode, style="pennylane", wire_options=wire_options)
    #drawer = qml.draw(qnode)
    print(drawer())
    
vqfe_results = vqfe_subroutine.vqfe_from_circuit(
    circuit_fn = circuit_fn,
    active_rho = active_rho, 
    active_rho_delta = active_rho_delta,
    L=2, 
    m=1, 
    maxiter=200)
    
vqfe_results


In [119]:
# plot fidelities

## parameters
N=4
n=3
trace_out_indices = np.random.choice(range(N), size= N - n, replace=False)
J=5
delta=0.1
m=1
# initial_state=None
DEBUG=False
derivative_delta=1e-3

results = []

for h_z in np.linspace(0, 1, 10):

    # classical fidelity
    # classical
    model = density_generator.SpinChainXXZ(n=N, J=J, h_z=h_z, initial_state="0", DEBUG=DEBUG) 
    rho_mixed, rho_delta_mixed = model.generate_mixed_density_matrices_with_perturbation(delta=delta,trace_out_indices=trace_out_indices)
    
    results.append({
        "hz": h_z,
        "uhlmann_fidelity": helper_functions.uhlmann_fidelity_root(rho_mixed, rho_delta_mixed),
        "pennylane_fidelity": helper_functions.fidelity_pennylane(rho_mixed, rho_delta_mixed)
    })

    # quantum fidelity
    

df = pd.DataFrame(results)

    

In [7]:
def example_demo():
    """
    Demonstration: 
    We'll pick n_qubits=1 for simplicity, 
    define two small density matrices for rho, rho_delta, 
    measure sub/super fidelities, and compute 
    approximate QFI bounds for a small delta=0.1.
    """
    n_qubits = 1

    # Example: a pure state for rho
    psi = np.array([1.0, 0.0])
    rho = np.outer(psi, psi)

    # Another pure state for rho_delta
    phi = np.array([np.sqrt(0.7), np.sqrt(0.3)])
    rho_delta = np.outer(phi, phi)

    # Measure sub- and super-fidelity
    E_val, R_val = quantum_subroutines_pennylane.measure_sub_super_fidelity(n_qubits, rho, rho_delta)
    print("Subfidelity E(rho, rho_delta) =", E_val)
    print("Superfidelity R(rho, rho_delta) =", R_val)

    # Bounds on the standard fidelity: sqrt(E) <= F <= sqrt(R).
    F_lower = np.sqrt(E_val)
    F_upper = np.sqrt(R_val)
    print("Lower bound on fidelity =", F_lower)
    print("Upper bound on fidelity =", F_upper)

    # For small parameter shift delta, eq. (3) in the paper gives:
    #   I_delta(rho) = (8 / delta^2)*(1 - F(rho, rho_delta)).
    # So using the sub- and super-fidelity as bounds on F,
    # we get bounds on the QFI:
    delta = 0.1
    I_lower = (8.0 / delta**2)*(1.0 - F_upper)
    I_upper = (8.0 / delta**2)*(1.0 - F_lower)
    print("Approx. Lower bound on QFI =", I_lower)
    print("Approx. Upper bound on QFI =", I_upper)

example_demo()

Subfidelity E(rho, rho_delta) = 0.6999999999999995
Superfidelity R(rho, rho_delta) = 0.7000000000000002
Lower bound on fidelity = 0.8366600265340752
Upper bound on fidelity = 0.8366600265340757
Approx. Lower bound on QFI = 130.67197877273944
Approx. Upper bound on QFI = 130.6719787727398


In [8]:
n = 5 ## numero finale di qubit
N = 7 ## numero di qubit prima della traccia parziale

h_z_s = np.linspace(0, 5, num=20) # spazio dei valori di h_z

a_x= 3

delta = 0.5
derivative_delta = 1e-5
m = 7
DEBUG=True
initial_state = helper_functions.random_mixed_density_matrix(N, n)

# ket = qutip.rand_ket(2**n, distribution="haar").full().flatten()

y_s = np.array([pipeline_v200.simulation(
                        N=N,
                        n=n,
                        initial_state= initial_state,
                        DEBUG=True,
                        derivative_delta=derivative_delta,
                        a_x= a_x, 
                        h_z= h_z, ## variable
                        delta = delta,
                        m = m) for h_z in h_z_s])

# just for plotting clarity
sparse_indices = np.array([i for i in list(range(len(h_z_s))) if i%10==0])

Generating random mixed density matrix
purity of initial state = (0.26981122154608717-8.74138001566438e-19j) 
Dimension of the Hilbert space: 32
Initial state type: <class 'numpy.ndarray'> | initial_state: [[ 0.02705162+4.75927742e-19j -0.0066634 -1.04648595e-03j
   0.00219118-8.08992609e-05j ...  0.00640547-3.48045042e-03j
   0.00103602-4.22397260e-03j  0.00293316+9.05987136e-03j]
 [-0.0066634 +1.04648595e-03j  0.03801386+5.42306155e-19j
  -0.00519149+1.57470705e-04j ...  0.004617  -4.57462584e-03j
  -0.00974733+1.88973953e-02j -0.00731824-5.04047077e-03j]
 [ 0.00219118+8.08992609e-05j -0.00519149-1.57470705e-04j
   0.01861122+1.05932863e-19j ... -0.00763026+4.78734553e-03j
   0.01822059-1.82303460e-02j -0.00737993-2.60657101e-03j]
 ...
 [ 0.00640547+3.48045042e-03j  0.004617  +4.57462584e-03j
  -0.00763026-4.78734553e-03j ...  0.01733077-1.36813895e-19j
  -0.01939275+5.62017750e-03j -0.01053872+3.33365784e-03j]
 [ 0.00103602+4.22397260e-03j -0.00974733-1.88973953e-02j
   0.01822059+1

purity of initial state = (0.26981122154608717-8.74138001566438e-19j) 
Dimension of the Hilbert space: 32
Initial state type: <class 'numpy.ndarray'> | initial_state: [[ 0.02705162+4.75927742e-19j -0.0066634 -1.04648595e-03j
   0.00219118-8.08992609e-05j ...  0.00640547-3.48045042e-03j
   0.00103602-4.22397260e-03j  0.00293316+9.05987136e-03j]
 [-0.0066634 +1.04648595e-03j  0.03801386+5.42306155e-19j
  -0.00519149+1.57470705e-04j ...  0.004617  -4.57462584e-03j
  -0.00974733+1.88973953e-02j -0.00731824-5.04047077e-03j]
 [ 0.00219118+8.08992609e-05j -0.00519149-1.57470705e-04j
   0.01861122+1.05932863e-19j ... -0.00763026+4.78734553e-03j
   0.01822059-1.82303460e-02j -0.00737993-2.60657101e-03j]
 ...
 [ 0.00640547+3.48045042e-03j  0.004617  +4.57462584e-03j
  -0.00763026-4.78734553e-03j ...  0.01733077-1.36813895e-19j
  -0.01939275+5.62017750e-03j -0.01053872+3.33365784e-03j]
 [ 0.00103602+4.22397260e-03j -0.00974733-1.88973953e-02j
   0.01822059+1.82303460e-02j ... -0.01939275-5.620177

In [ ]:
import density_generator

# from qiskit.visualization import plot_state_city
from importlib import reload
import pandas as pd
import numpy as np
import classicalQFI
import qutip
import helper_functions

reload(helper_functions)
reload(density_generator)
reload(classicalQFI)



def simulation(
    N=10,
    n=7,
    a_x=1,
    h_z=0.1,
    delta=0.01,
    m=1,
    initial_state=None,
    DEBUG=False,
    derivative_delta=1e-3,
):
    """ "
    Simulate the quantum state evolution and compute the QFI bounds.
    params:
    N: number of qubits
    n: ending number of qubits (we will trace out N - n qubits)
    a_x: coupling coefficient for the Ising interaction term
    h_z: perturbation strength
    delta: "error" state difference
    m: number of largest eigenvalues to keep
    initial_state: initial state of the system (matrix or state vector)
    DEBUG: print debug information
    derivative_delta: step size for SLD
    """

    if initial_state is None:
        if DEBUG:
            print("Generating random initial state")
        # Generate random initial state
        ket = helper_functions.random_haar_ket(2**N)

        rho = np.outer(ket, np.conj(ket))

        trace_out_index = np.random.choice(range(N), size=N - n, replace=False)

        v = helper_functions.trace_out(rho=rho, trace_out_index=trace_out_index)
    else:
        v = initial_state

    if DEBUG:
        purity = np.trace(v @ v)
        print(f"purity of initial state = {purity} ")

    # Generate new density matrix using Ising model
    model = density_generator.IsingQuantumState(
        n=n, a_x=a_x, h_z=h_z, initial_state=v, DEBUG=DEBUG
    )
    rho = model.generate_density_matrix()

    if DEBUG:
        # Calculate purity and participation ratio
        purity = np.trace(rho @ rho)
        participation_ratio = 1 / purity
        print(f"Purity of evolved state: {purity}")
        print(f"Participation ratio: {participation_ratio}")

    ####### END) generation of the matrices

    rho, rho_delta = model.generate_density_matrices_with_perturbation(delta=delta)

    ## Getting results from classical QFI
    results = classicalQFI.compute_tqfi_bounds(
        rho=rho, rho_delta=rho_delta, m=m, delta=delta
    )

    if derivative_delta is None:
        derivative_delta = (delta / 100,)

    # this method is not parameter agnostic, so it is not into classicalQFI
    qfi_from_SLD = model.compute_qfi_with_sld(delta=delta, d=derivative_delta)

    # append the true QFI to the results, taken using the pure states,
    # before the partial trace (this idea was not good)

    ## Optional overlap with |00...00>
    # ket_0n = np.zeros(n-1, dtype=complex)
    # ket_0n[0] = 1.0
    # ket_0n = ket_0n.transpose()
    # overlap = ket_0n @

    if DEBUG:  ## check trace of rho
        # print(f"trace of rho: {np.trace(rho)}")
        # print(f"trace of rho + delta: {np.trace(rho_delta)}")
        ## check also purity
        # print(f"purity of rho: {np.trace(rho @ rho)}")
        # print(f"purity of rho + delta: {np.trace(rho_delta @ rho_delta)}")
        ## check also rank
        # print(f"rank of rho: {np.linalg.matrix_rank(rho)}")
        # print(f"rank of rho + delta: {np.linalg.matrix_rank(rho_delta)}")
        # put all of the above in a dictionary
        results["trace_rho"] = np.trace(rho)
        results["trace_rho_delta"] = np.trace(rho_delta)
        results["purity_rho"] = np.trace(rho @ rho)
        results["purity_rho_delta"] = np.trace(rho_delta @ rho_delta)
        results["rank_rho"] = np.linalg.matrix_rank(rho)
        results["rank_rho_delta"] = np.linalg.matrix_rank(rho_delta)

        results["truncated_eigenvalues"] = (
            helper_functions.get_truncated_eigen_decomposition(rho=rho, m=m)[0]
        )
        results["eigenvalues"] = helper_functions.compute_eigen_decomposition(rho=rho)[
            0
        ]

    results["QFI_from_SLD"] = qfi_from_SLD

    return results
